In [3]:
import os
import re
import zipfile
from pycocotools.coco import COCO
import numpy as np
import tensorflow as tf
from tensorflow import keras

from PIL import Image
import matplotlib
%matplotlib inline
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import skimage.io
import skimage.transform
import skimage.color
import skimage
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import os
from sklearn.model_selection import train_test_split
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
#train_list
dataDir='..'
dataType='train2017'
annFile='{}/datasets/coco_2017/annotations/instances_{}.json'.format(dataDir, dataType)
coco=COCO(annFile)
root_dir='../datasets/coco_2017/images/'
set_name='train2017_resize224/'


loading annotations into memory...


In [3]:
#train_ids
categories = coco.loadCats(coco.getCatIds())
categories.sort(key=lambda x: x['id'])
image_ids = coco.getImgIds()
len(image_ids)

118287

In [4]:
#val_list
dataDir='..'
dataType2='val2017'
annFile='{}/datasets/coco_2017/annotations/instances_{}.json'.format(dataDir, dataType2)
coco2=COCO(annFile)
root_dir2='../datasets/coco_2017/images/'
set_name2='val2017_resize224/'

loading annotations into memory...
Done (t=0.30s)
creating index...
index created!


In [5]:
#val_ids
categories2 = coco2.loadCats(coco2.getCatIds())
categories2.sort(key=lambda x: x['id'])
image_ids2 = coco2.getImgIds()
len(image_ids2)

5000

In [5]:
#category = 80
category_dict = {(1,0), (2,1), (3,2), (4,3), (5,4), (6,5), (7,6), (8,7), (9,8), (10,9), (11,10), (13,11), (14, 12), (15, 13), (16, 14), (17, 15), (18, 16), (19, 17),
(20, 18), (21, 19), (22, 20), (23, 21), (24, 22) , (25, 23), (27, 24), (28, 25), (31, 26), (32, 27), (33, 38), (34, 29), (35, 30), (36, 31), (37, 32), (38,33), (39, 34),
(40, 35), (41, 36), (42, 37), (43 ,38), (44 , 39), (46 , 40), (47 ,41), (48,42), (49 ,43), (50 , 44), (51, 45), (52, 46),  (53, 47), (54 , 48), (55 , 49), (56, 50), (57 , 51),
(58 , 52), (59 , 53), (60, 54), (61, 55), (62, 56), (63, 57), (64, 58), (65, 59), (67, 60), (70 , 61), (72, 62), (73 , 63), (74, 64), (75 , 65), (76 , 66), (77 , 67), (78 , 68),
(79 , 69),(80 , 70), (81 , 71), (82 , 72), (84 , 73), (85 , 74), (86 , 75), (87, 76), (88 , 77), (89 , 78), (90 , 79)}

In [6]:
#category = 10
category_dict2 = {(1,0), (2,1), (3,1), (4,1), (5,1), (6,1), (7,1), (8,1), (9,1), (10,2), (11,2), (13,2), (14, 2), (15, 2), (16, 2), (17, 2), (18, 2), (19, 2),
(20, 2), (21, 2), (22, 2), (23, 2), (24, 2) , (25, 2), (27, 3), (28, 3), (31, 3), (32, 3), (33, 3), (34, 3), (35, 3), (36, 3), (37, 3), (38,3), (39, 3),
(40, 3), (41, 3), (42, 3), (43 ,3), (44 , 4), (46 , 4), (47 ,4), (48,4), (49 ,4), (50 , 4), (51, 4), (52, 5),  (53, 5), (54 , 5), (55 , 5), (56, 5), (57 , 5),
(58 , 5), (59 , 5), (60, 5), (61, 5), (62, 6), (63, 6), (64, 6), (65, 6), (67, 6), (70 , 6), (72, 7), (73 , 7), (74, 7), (75 , 7), (76 , 7), (77 , 7), (78 , 8),
(79 , 8),(80 , 8), (81 , 8), (82 , 8), (84 , 9), (85 , 9), (86 , 9), (87, 9), (88 , 9), (89 , 9), (90 , 9)}


In [4]:

def create_imagelist(num: int):
    img_list = []
    labels = []
    index_list = []
    for i in range(num):
    
        image_info = coco.loadImgs(image_ids[i])[0]
        path = os.path.join(root_dir, set_name, image_info['file_name'])
        img = Image.open(path)
        imgarray = np.array(img)
        annotations_ids =coco.getAnnIds(imgIds=image_ids[i], iscrowd=False)
        if imgarray.ndim == 3 and annotations_ids:
                
                img_list.append(imgarray)
                index_list.append(i)
                coco_annotations = coco.loadAnns(annotations_ids)
                areas =[]
                areas.append(x['area'] for x in coco_annotations)
                index = np.argmax(areas)
                category_id = coco_annotations[index]['category_id']
    
                for c,d in category_dict:
                    if c == category_id:
                        label = [0 for i in range(80)]
                        label[d] = 1
                labels.append(label)  
    return np.asarray(img_list), np.asarray(labels)
                


In [7]:

def create_multi_imagelist(num: int):
    img_list = []


    labels_list= []
    index_list = []
    for i in range(num):
        
        image_info = coco.loadImgs(image_ids[i])[0]
        path = os.path.join(root_dir, set_name, image_info['file_name'])
        img = Image.open(path)
        imgarray = np.array(img)
        annotations_ids =coco.getAnnIds(imgIds=image_ids[i], iscrowd=False)
        label =[]
        if imgarray.ndim == 3 and annotations_ids:
                
                img_list.append(imgarray)
                
                coco_annotations = coco.loadAnns(annotations_ids)
                
                label = [0 for i in range(80)]
                for index in range(len(coco_annotations)):
                    
                    category_id = coco_annotations[index]['category_id']
                
                    for c,d in category_dict:
                        if c == category_id:
                            label[d] = 1
                labels_list.append(label)
        
                    
    return np.asarray(img_list), np.asarray(labels_list)
            


In [34]:
A = create_multi_imagelist(10000)[1]
A_list = []
for a in A:
    A_list.append(np.count_nonzero(a == 1))
np.mean(A_list)

2.878818429277145

In [35]:

def create_val_imagelist(num: int):
    img_list = []
    labels = []
    index_list = []
    for i in range(num):
    
        image_info = coco2.loadImgs(image_ids2[i])[0]
        path = os.path.join(root_dir2, set_name2, image_info['file_name'])
        img = Image.open(path)
        imgarray = np.array(img)
        annotations_ids =coco2.getAnnIds(imgIds=image_ids2[i], iscrowd=False)
        if imgarray.ndim == 3 and annotations_ids:
                
                img_list.append(imgarray)
                index_list.append(i)
                coco_annotations = coco2.loadAnns(annotations_ids)
                areas =[]
                areas.append(x['area'] for x in coco_annotations)
                index = np.argmax(areas)
                category_id = coco_annotations[index]['category_id']
    
                for c,d in category_dict:
                    if c == category_id:
                        label = [0 for i in range(80)]
                        label[d] = 1
                labels.append(label)  
    return np.asarray(img_list), np.asarray(labels)
                

In [8]:

def create_multi_val_imagelist(num: int):
    img_list = []
    labels_list = []
    for i in range(num):
    
        image_info = coco2.loadImgs(image_ids2[i])[0]
        path = os.path.join(root_dir2, set_name2, image_info['file_name'])
        img = Image.open(path)
        imgarray = np.array(img)
        annotations_ids =coco2.getAnnIds(imgIds=image_ids2[i], iscrowd=False)
        if imgarray.ndim == 3 and annotations_ids:
                
                img_list.append(imgarray)
               
                coco_annotations = coco2.loadAnns(annotations_ids)
                
                
                label = [0 for i in range(80)]
                for index in range(len(coco_annotations)):
                    
                    category_id = coco_annotations[index]['category_id']
                    for c,d in category_dict:
                        if c == category_id:
                            label[d] = 1
                labels_list.append(label)
              
    return np.asarray(img_list), np.asarray(labels_list)
                

In [9]:
X, Y = create_multi_imagelist(60000)
X.shape, Y.shape

((59472, 224, 224, 3), (59472, 80))

In [17]:
X_test, Y_test = create_multi_val_imagelist(5000)
X_test.shape, Y_test.shape

((4952, 224, 224, 3), (4952, 80))

In [6]:
X, Y = create_imagelist(10000)
X.shape, Y.shape

NameError: name 'coco' is not defined

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=10,figsize=(20,20))
for i in range(10):
    ax[i].imshow(X[i+3000])
    ax[i].axis('off')
    for c,d in category_dict:
        if d == np.argmax(Y[i+3000]):
            print(c, end = ' ')

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen= ImageDataGenerator(featurewise_center=True, fill_mode="nearest")
# train = datagen.flow(X, Y, batch_size=64)
# len(train.next()[0])


In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=10,figsize=(20,20))
for i in range(10):
    ax[i].imshow(X[i])
    ax[i].axis('off')
    
    c_list = []
    for x in np.where(Y[i] == 1):
        for c,d in category_dict:
            if d == x:
                print(c, end= ' ')
    
            
     


In [10]:
def hard_swish(features):

  
  features = tf.convert_to_tensor(features)
  fdtype = features.dtype
  return features * tf.nn.relu6(features + tf.cast(3., fdtype)) * (1. / 6.)


In [11]:
def se_block(inputs, ch, ratio=16):

    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Dense(ch//ratio, activation='relu')(x)
    y = tf.keras.layers.Dense(ch, activation='sigmoid')(x)
    y = tf.keras.layers.multiply([inputs, y])
    out = hard_swish(y)
    return out



In [12]:
def dep_bn(inputs, filter : int, kernel :int, stride: int):

  y = tf.keras.layers.Conv2D(filters = filter, kernel_size= kernel, strides= stride, padding= 'same')(inputs)
  bn = tf.keras.layers.BatchNormalization(axis = 1)(y)
  relu = tf.nn.relu6(bn)
  
  return relu 


def Inverted_residual_SEblock(x, filter : int, kernel: int, strides = int):


    #pointwise
    y = tf.keras.layers.Conv2D(filters = filter, kernel_size=  1, strides = strides, padding= 'same')(x)
    bn = tf.keras.layers.BatchNormalization(axis = 1)(y)
    y_p = tf.nn.relu6(bn)
    
    
    #depthwise
    y = tf.keras.layers.DepthwiseConv2D(kernel_size=  kernel, strides= strides, padding= 'same', depth_multiplier = 1)(y_p)
    bn = tf.keras.layers.BatchNormalization(axis = 1)(y)
    y_d = tf.nn.relu6(bn)
   
   
    #pointwise
    y = tf.keras.layers.Conv2D(filters = filter, kernel_size= 1, strides=strides, padding= 'same')(y)
    y = tf.keras.layers.BatchNormalization(axis = 1)(y)
    out = tf.keras.layers.ReLU()(y)

    if strides == 1:
        if x.shape[3] != filter : 
           x = tf.keras.layers.Conv2D(filters = filter, kernel_size = 1, strides = strides, padding = "same")(x)
        out  = tf.keras.layers.Add()([x, out])
   
    y = se_block(out, filter)

    #pointwise
    y = tf.keras.layers.Conv2D(filters = filter, kernel_size= 1, strides=strides, padding= 'same')(y)
    y = tf.keras.layers.BatchNormalization(axis = 1)(y)
    out = tf.nn.relu6(y)


    return out


In [13]:
def create_mobilenetV3(inputs):

   x = dep_bn(inputs, 16, 3, 2)
   x = Inverted_residual_SEblock(x, 16, 3, 1)
   x = Inverted_residual_SEblock(x, 16, 3, 2)
   x = Inverted_residual_SEblock(x, 24, 3, 1)
   x = Inverted_residual_SEblock(x, 24, 5, 2)
   x = Inverted_residual_SEblock(x, 40, 5, 1)
   x = Inverted_residual_SEblock(x, 40, 5, 1)
   x = Inverted_residual_SEblock(x, 40, 3, 2)
   x = Inverted_residual_SEblock(x, 80, 3, 1)
   x = Inverted_residual_SEblock(x, 80, 3, 1)
   x = Inverted_residual_SEblock(x, 80, 3, 1)
   x = Inverted_residual_SEblock(x, 80, 3, 1)
   x = Inverted_residual_SEblock(x, 112, 3, 1)
   x = Inverted_residual_SEblock(x, 112, 5, 1)
   x = Inverted_residual_SEblock(x, 160, 5, 1)
   x = dep_bn(x, 160, 1, 1)
   
   t = tf.keras.layers.AveragePooling2D(1)(x)
   t = tf.keras.layers.Flatten()(t)
   outputs = tf.keras.layers.Dense(80, activation='sigmoid')(t)
   return outputs

In [14]:
inputs = tf.keras.Input(shape=(224,224,3))
outputs = create_mobilenetV3(inputs)

model = tf.keras.Model(inputs, outputs)

opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

In [30]:
model.save("model0712")

/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: model0712/assets


In [19]:
def making_ypred(predictions):
    pred2s = []
    for pred in predictions:
        pred2 = np.argsort(pred)[::-1]
        for i in range(len(pred2)):
            if pred2[i] < 3:
                pred2[i] = 1
                
            else:
                pred2[i] = 0
        pred2s.append(pred2)
    return pred2s

predictions = model.predict(X_test)
y_pred = np.asarray(making_ypred(predictions))
y_pred.shape

(4952, 80)

In [16]:
y_pred = model.predict(X_test)[97]
y_pred2= np.argsort(y_pred)[::-1]
print(np.argmax(y_pred))
y_pred, y_pred2
for i in range(len(y_pred2)):
    if y_pred2[i] < 3:
        y_pred2[i] = 1
    else:
        y_pred2[i] = 0
y_pred2

        

NameError: name 'X_test' is not defined

In [20]:
Y_test.shape

(4952, 80)

In [3]:
model2 = keras.models.load_model("model0713-1")

In [15]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.54      1.00      0.70      2693
           1       0.00      0.00      0.00       149
           2       0.25      0.62      0.35       535
           3       0.00      0.00      0.00       159
           4       0.00      0.00      0.00        97
           5       0.00      0.00      0.00       189
           6       0.00      0.00      0.00       157
           7       0.00      0.00      0.00       250
           8       0.00      0.00      0.00       121
           9       0.00      0.00      0.00       191
          10       0.00      0.00      0.00        86
          11       0.00      0.00      0.00        69
          12       0.00      0.00      0.00        37
          13       0.00      0.00      0.00       235
          14       0.00      0.00      0.00       125
          15       0.00      0.00      0.00       184
          16       0.00      0.00      0.00       177
          17       0.00    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, Y_pred).shape)
confusion_matrix(y_true, Y_pred)

(10, 10)


array([[178, 205, 350,   0,  10,  45, 164,   0,   0,   0],
       [105, 212, 444,   0,   3,  11,  33,   0,   0,   0],
       [139,  68, 701,   0,  12,  35, 119,   0,   0,   0],
       [ 97,  60, 197,   0,   3,  13,  67,   0,   0,   0],
       [ 36,  11,  81,   0,   7, 101, 132,   0,   0,   0],
       [ 18,   3,  50,   0,   4, 117,  30,   0,   0,   0],
       [ 78,  20, 118,   0,   9,  57, 217,   0,   0,   0],
       [ 36,  10,  77,   0,   3,  11, 145,   0,   0,   0],
       [ 11,   2,  13,   0,   0,   5,  55,   0,   0,   0],
       [ 35,  35,  75,   0,   1,  38,  40,   0,   0,   0]])

In [21]:
model.save("model0709")

/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: model0709/assets


In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen= ImageDataGenerator(featurewise_center=True, fill_mode="nearest")
train = datagen.flow(X, Y, batch_size=64)
len(train.next()[0])

model.fit(datagen.flow(X, Y, batch_size=100), validation_data = datagen.flow(X_test, Y_test,
         batch_size=32), steps_per_epoch=len(X) / 100, epochs=1)
    

In [16]:

class DataGen(tf.keras.utils.Sequence):
    
    def __init__(self, coco, root_dir, set_name, image_ids, batch_size = 64 , input_size= (224, 224, 3),
                 shuffle=True):
        
        self.image_ids = image_ids
        self.indexes = np.arange(len(self.image_ids))
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.root_dir = root_dir
        self.set_name = set_name
        self.coco = coco
       
        
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __get_input(self, image_index):
        
        
        image_info = coco.loadImgs(image_ids[image_index])[0]
        path = os.path.join(self.root_dir, self.set_name, image_info['file_name'])
        img = Image.open(path)
        imgarray = np.array(img)
        annotations_ids =coco.getAnnIds(imgIds=image_ids[image_index], iscrowd=False)
  
        if imgarray.ndim == 3 and annotations_ids:
             
                
                coco_annotations = coco.loadAnns(annotations_ids)
                
                label = [0 for i in range(10)]
                for index in range(len(coco_annotations)):
                    
                    category_id = coco_annotations[index]['category_id']
                
                    for c,d in category_dict2:
                        if c == category_id:
                            label[d] = 1
               
                return imgarray, label
    
    def __getoutput__(self):
        
        X = []
        Y = []
        for image_index in range(len(self.image_ids)):
             if self.__get_input(image_index) is not None :
                 X.append(self.__get_input(image_index)[0])
                 Y.append(self.__get_input(image_index)[1])
                
        return np.asarray(X), np.asarray(Y)
        
    def __getitem__(self, idx):
         
        X, Y = self.__getoutput__()
        
        
        batch_x = X[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = Y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return batch_x, batch_y
    
    def __len__(self):
          return int(np.floor(len(self.image_ids) / self.batch_size))

In [ ]:
train = datagen.flow(X, Y, batch_size=64)
model.fit(datagen.flow(X, Y, batch_size=64), validation_data = datagen.flow(X_test, Y_test,
         batch_size=64), steps_per_epoch=len(X) / 64, epochs=10)
    

In [32]:
model.predict(X_test)[1]

array([0.09642872, 0.00162802, 0.01660121, 0.01321321, 0.16993096,
       0.41670942, 0.20290275, 0.02421168, 0.02415303, 0.03422097],
      dtype=float32)

In [33]:
Y_test[1]

array([0, 0, 0, 0, 0, 1, 1, 0, 1, 0])

In [ ]:
train = datagen.flow(X, Y, batch_size=64)
len(train.next()[0])

len(train)


In [ ]:

model.fit(datagen.flow(X, Y, batch_size=100), validation_data = datagen.flow(X_test, Y_test,
         batch_size=32), steps_per_epoch=len(X) / 100, epochs=1)
    



In [17]:
image_ids = coco.getImgIds()
image_ids2 = coco2.getImgIds()
train_generator = DataGen(image_ids = image_ids[:50000], root_dir='../datasets/coco_2017/images/', 
        set_name='train2017_resize224/', coco= coco) 

validation_generator = DataGen(image_ids = image_ids2, root_dir='../datasets/coco_2017/images/', 
        set_name='val2017_resize224/', coco = coco2) 

len(train_generator), len(validation_generator)

(781, 78)

In [ ]:
compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / 64))
model.fit_generator(generator=train_generator,
         validation_data=validation_generator,   epochs = 5, use_multiprocessing=True,
                    workers=6)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5


In [22]:
import math

In [14]:
X = np.asarray(X)
Y = np.asarray(Y)
X.shape, Y.shape

((987, 224, 224, 3), (987, 10))

In [13]:
X = []
Y = []
for image_index in range(len(image_ids[:1000])):
    
    if __get_input(image_index) is not None:
        X.append(__get_input(image_index)[0])
        Y.append(__get_input(image_index)[1])




In [1]:
def __get_input(image_index):
        
        
        image_info = coco.loadImgs(image_ids[image_index])[0]
        path = os.path.join(root_dir, set_name, image_info['file_name'])
        img = Image.open(path)
        imgarray = np.array(img)
        annotations_ids =coco.getAnnIds(imgIds=image_ids[image_index], iscrowd=False)
    
            
        if imgarray.ndim == 3 and annotations_ids:
             
                
                coco_annotations = coco.loadAnns(annotations_ids)
                
                label = [0 for i in range(10)]
                for index in range(len(coco_annotations)):
                    
                    category_id = coco_annotations[index]['category_id']
                
                    for c,d in category_dict2:
                        if c == category_id:
                            label[d] = 1
               
                return imgarray, label
                
                